In [1]:
!pip install --quiet "git+https://github.com/optimizacion-2-2021-1-gh-classroom/practica-2-segunda-parte-caroacostatovany.git#egg=mex&subdirectory=src"

In [2]:
#pip uninstall mex -y

In [3]:
import numpy as np
import timeit
import scipy.io as sio
import datetime
import os

# mex
from mex.simplex.simplex_networks import create_matrix, pivots_col, pivots_row, \
find_negative_col, find_negative_row, find_pivot_col, find_pivot_row, pivot
from mex.simplex.problem_definition import add_cons, \
constrain, add_obj, obj, maxz, minz

# scipy
from pytest import approx
from scipy.optimize import linprog

In [4]:
def generate_tableau(A, b, blow, bupp, c, compr = True):
    """
    Genera la matriz bajo el supuesto de que se tiene ya capturado los datos de entrada. Bajo el problema del tipo:
    min/max c^Tx
    Ax = b
    Concantenando renglones de A y juntando los limites de las restricciones (multiplicando por -1 donde sea necesario)
    """
    
    if compr:
        A = A.todense()
        A = np.array(A)
    
    n_restr, n_vars = A.shape
    
    
    # Caso en que blow = 0, esto es, no es ninguna restriccion bajo los supuestos de simplex
    if (blow.sum() == 0) and (min(blow)==0) and False:
        # Concatenar A con matrices identidades de variables sinteticas de forma rbind
        identidad_vars_sint = np.eye(n_vars, n_vars)
        c = c.T
        A_new = np.concatenate((A, identidad_vars_sint, c), axis=0)
    
        # Concatenar variables coeficientes sinteticas
        A_sint = np.eye(n_restr + 1*n_vars + 1, n_restr + 1*n_vars + 1)
        A_sint[0:n_restr,:] = A_sint[0:n_restr,:]*0
        
        # Concatenar A_new con A_sint de formca rbind
        A_new = np.c_[A_new, A_sint]
        
        # Juntar cotas de restricciones
        b_new = np.concatenate((b, bupp, np.array([[0]])), axis=0)
        
        # Concatenar A_new, b_new de forma cbind
        A_new = np.c_[A_new, b_new]
    
    else:
        # Concatenar A con matrices identidades de variables sinteticas de forma rbind
        identidad_vars_sint = np.eye(n_vars, n_vars)
        c = c.T
        A_new = np.concatenate((A, identidad_vars_sint, -identidad_vars_sint, c), axis=0)
    
        # Concatenar variables coeficientes sinteticas
        A_sint = np.eye(n_restr + 2*n_vars + 1, n_restr + 2*n_vars + 1)
        A_sint[0:n_restr,:] = A_sint[0:n_restr,:]*0
        
        # Concatenar A_new con A_sint de formca rbind
        A_new = np.c_[A_new, A_sint]
        
        # Juntar cotas de restricciones
        b_new = np.concatenate((b, bupp, -blow, np.array([[0]])), axis=0)
        
        # Concatenar A_new, b_new de forma cbind
        A_new = np.c_[A_new, b_new]
    
    return A_new

# Prueba de función `generate_tableau`

In [5]:
matrix_min_approx = create_matrix(2,7)
matrix_min_approx

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [6]:
constrain(matrix_min_approx,'1,1,E,6')
constrain(matrix_min_approx,'-1,2,E,6')
constrain(matrix_min_approx,'4,3,E,6')
constrain(matrix_min_approx,'1,0,G,0')
constrain(matrix_min_approx,'0,1,G,0')
constrain(matrix_min_approx,'1,0,L,18')
constrain(matrix_min_approx,'0,1,L,18')
obj(matrix_min_approx,'-1,-3,0')
matrix_min_approx

array([[ 1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6.],
       [-1.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6.],
       [ 4.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6.],
       [-1., -0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0., -0.],
       [-0., -1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0., -0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0., 18.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0., 18.],
       [ 1.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.]])

In [7]:
A_prueba = np.array([[1,1], [-1,2], [4,3]])
b_prueba = np.array([[6,6,6]]).T
c_prueba = np.array([[1,3]]).T
blow_prueba = np.array([[0,0]]).T
bupp_prueba = np.array([[18,18]]).T

In [8]:
tableau_prueba = generate_tableau(A_prueba, b_prueba, blow_prueba, bupp_prueba, c_prueba, False)
tableau_prueba

array([[ 1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6.],
       [-1.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6.],
       [ 4.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6.],
       [ 1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0., 18.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0., 18.],
       [-1., -0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [-0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 1.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.]])

# Prueba del paquete `mex`

In [9]:
#mat = sio.loadmat('../data/AFIRO.mat')
mat = sio.loadmat('../data/25FV47.mat')

In [10]:
A = mat['A']
b = mat['b']
lbounds = mat['lbounds']
ubounds = mat['ubounds']
c = mat['c']

In [11]:
tableau = generate_tableau(A, b, lbounds, ubounds, c)

In [12]:
print("Inicio a correr a las: ")
now = datetime.datetime.now()
print(now.year,"-", now.month,"-", now.day,":\_o_/ Hora \_o_/:", now.hour,":", now.minute,":", now.second)

Inicio a correr a las: 
2021 - 5 - 15 :\_o_/ Hora \_o_/: 19 : 41 : 6


In [13]:
problem_approx = minz(tableau)

Valor objetivo 0.0


/home/ubuntu/.local/lib/python3.8/site-packages/mex/simplex/problem_definition.py:261: RuntimeWarning: invalid value encountered in double_scalars
  cambio = abs(z_0 - z_t) / z_t


Valor objetivo 1.155872586397737
Valor objetivo 1.851164194116168
Valor objetivo 0.9030016584546501
Valor objetivo -14.777204974195582
Valor objetivo -63.55407142857141
Valor objetivo -63.554071428571405
Valor objetivo -14.777204974195598
Valor objetivo -63.55407142857142
Valor objetivo -63.55407142857141
Valor objetivo -63.55407142857141
Valor objetivo -63.55407142857141
Valor objetivo -63.55407142857141
Valor objetivo -63.55407142857141
Valor objetivo -63.554071428571405
Valor objetivo -61.88814342857142
Valor objetivo -61.88814342857142
Valor objetivo -56.00976925443145
Valor objetivo -44.14401383346687
Valor objetivo -43.609534813186784
Valor objetivo -41.653257949738816
Valor objetivo -36.299347710097166
Valor objetivo -35.73248519030206
Valor objetivo -34.15174830942342
Valor objetivo -33.12818860731103
Valor objetivo -33.12818860731103
Valor objetivo -33.12818860731103
Valor objetivo -33.409257541199
Valor objetivo -33.862600517397745
Valor objetivo -33.12818860731104
Valor obje

TypeError: 'NoneType' object is not subscriptable

In [14]:
problem_approx['min']

NameError: name 'problem_approx' is not defined

In [21]:
tableau_prueba = tableau

In [22]:
n_restr = tableau_prueba.shape[0]-1
n_vars = A.shape[1]

In [24]:
c_min_obj = tableau_prueba[-1,0:n_vars]

In [25]:
A_min_obj = tableau_prueba[0:n_restr, 0:n_vars]

In [26]:
b_min_obj = tableau_prueba[0:n_restr, -1]

In [71]:
min_obj = linprog(c, A_ub = A, b_ub = b).fun
print(min_obj)

<ipython-input-71-1f7e740222bd>:1: OptimizeWarning: Sparse constraint matrix detected; setting 'sparse':True.
  min_obj = linprog(c, A_ub = A, b_ub = b).fun


-637721020464.894


In [40]:
tableau_prueba = generate_tableau(A_prueba, b_prueba, blow_prueba, bupp_prueba, c_prueba, False)
tableau_prueba

array([[ 1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6.],
       [-1.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6.],
       [ 4.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6.],
       [ 1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0., 18.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0., 18.],
       [-1., -0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [-0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 1.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.]])

In [67]:
A_new = np.concatenate(( np.array(A.todense()), c.T), axis=0)

b_new = np.concatenate((b, np.array([[0]])), axis=0)

A_new = np.c_[A_new, np.eye(A.shape[0]+1, A.shape[0]+1)]

A_new = np.c_[A_new, b_new]

In [69]:
problem_approx = minz(A_new)

Valor objetivo 0.0
##################################
 Terminó pivoteo de columnas

Valor objetivo 19.2
Valor objetivo 919.2
Valor objetivo 1719.2
Valor objetivo 1959.2
Valor objetivo 2309.2
Valor objetivo 2959.2
Valor objetivo 3559.2
Valor objetivo 3699.6454999999996
Valor objetivo 3773.3614999999995
Valor objetivo 3850.1489999999994
Valor objetivo 3874.7209999999995
Valor objetivo 3890.0784999999996
Valor objetivo 3892.358591954023
Valor objetivo 3898.0855062972587
Valor objetivo 3899.4827078823587
##################################
 Terminó pivoteo de renglones



In [70]:
problem_approx['min']

-3899.4827078823587